# Doing frequentist statistics with Scipy  
  
### PyData DC 2016  
  
*Gustavo A. Patino*  
*Department of Biomedical Sciences*  
*Department of Neurology*  
*Oakland University William Beaumont School of Medicine*  
*Rochester, MI*  
  
patino@oakland.edu  
https://github.com/gapatino/Doing-frequentist-statistics-with-Scipy

**Iris Dataset:**  
Fisher RA. The use of multiple measurements in taxonomic problems. *Annals of Eugenics* 1936; 7 (2): 179–188

https://github.com/gapatino/Doing-frequentist-statistics-with-Scipy

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd

from tkinter import filedialog

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# Use file browser to find name and path of the CSV file that contains the dataset
data_file = filedialog.askopenfilename()
print(data_file)

### Loading the dataset

In [ ]:
dataset = pd.read_csv(data_file, sep=',', na_values=["."," ","na"]) # Can use lists for possible missing values
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset.head(n=10)

### Exploratory analysis

In [ ]:
dataset.describe(percentiles=[.25, .5, .75], include='all')

In [ ]:
grouped_iris = dataset.groupby('Type')
grouped_iris.mean()

#### Exercise  
Create a table that describes the grouped_iris data frame, including the 25th, 50th, and 75th percentiles

In [ ]:
grouped_iris.describe(percentiles=[.25, .5, .75], include='all')

In [ ]:
grouped_iris['Petal_Width'].hist()

In [ ]:
sns.pairplot(dataset, kind='reg', hue='Type')

### Data Extraction
For the statistical functions we will have to specify the dependent data variable and the independent data variable as two separate arrays

In [ ]:
grouped_iris['Type'] # .groupby function returns a GroupBy object that is lazily executed

In [ ]:
dataset[dataset['Type']=='setosa'].head(n=10) # Better to use the original dataset

#### Exercise  
How would you make separate variables containing the petal and sepal characteristics of each iris type?

In [ ]:
dataset[dataset['Type']=='setosa']['Petal_Length'].head(n=10)

In [ ]:
pl_setosa = dataset[dataset['Type']=='setosa']['Petal_Length']
pl_virginica = dataset[dataset['Type']=='virginica']['Petal_Length']
pl_versicolor = dataset[dataset['Type']=='versicolor']['Petal_Length']

pw_setosa = dataset[dataset['Type']=='setosa']['Petal_Width']
pw_virginica = dataset[dataset['Type']=='virginica']['Petal_Width']
pw_versicolor = dataset[dataset['Type']=='versicolor']['Petal_Width']

sl_setosa = dataset[dataset['Type']=='setosa']['Sepal_Length']
sl_virginica = dataset[dataset['Type']=='virginica']['Sepal_Length']
sl_versicolor = dataset[dataset['Type']=='versicolor']['Sepal_Length']

sw_setosa = dataset[dataset['Type']=='setosa']['Sepal_Width']
sw_virginica = dataset[dataset['Type']=='virginica']['Sepal_Width']
sw_versicolor = dataset[dataset['Type']=='versicolor']['Sepal_Width']

In [ ]:
type(sw_setosa)

In [ ]:
plt.hist(sw_versicolor, label='Versicolor', alpha=0.5)
plt.hist(sw_virginica, label='Virginica', alpha=0.5)
plt.hist(sw_setosa, label='Setosa', alpha=0.5)
plt.legend(loc='best')

### Normality Testing

In [ ]:
# Kolmogorov-Smirnov test: Fairly conservative
ks_pl_setosa = stats.kstest(pl_setosa, 'norm', mode='asymp') # mode opts: 'approx'. Dist can be any in scipy.stats
ks_pl_setosa

In [ ]:
# Shapiro test
shapiro_pw_setosa = stats.shapiro(pw_setosa)
shapiro_pw_setosa

In [ ]:
# Normal test: Combines skew and kurtosis measurement. Allows management of NaN
nt_sl_setosa = stats.normaltest(sl_setosa, nan_policy='omit') #nan_policy opts: 'propagate', 'raise'
nt_sl_setosa

In [ ]:
# Anderson test: Modified KS, returns critical values for a list of significance levels
anderson_sw_setosa = stats.anderson(sw_setosa, dist='norm')
anderson_sw_setosa

#### Exercise  
- What do the outputs mean?
- How would you extract only the *p*-value of a given test?

In [ ]:
print('KS: ', ks_pl_setosa)
print('Shapiro: ', shapiro_pw_setosa)
print('Normal: ', nt_sl_setosa)
print('Anderson: ', anderson_sw_setosa)

All of the outputs are the test value and associated *p*-value, except for Anderson test in which the test value is provided along with a table of critical values for given significances

In [ ]:
print('KS p-value using index: ', ks_pl_setosa[1])
# or
_ , p_ks_pl_setosa = stats.kstest(pl_setosa, 'norm', mode='asymp')
print('KS p-value using multiple variable assignment: ', p_ks_pl_setosa)

### Homogeneity of Variance

In [ ]:
# Bartlett test: Requires normal populations
bartlett_length_versicolor = stats.bartlett(pl_versicolor, sl_versicolor)
print(bartlett_length_versicolor)

In [ ]:
# Levene test: more robust than Bartlett if samples are non-normal. Can define what central tendency measure is used
levene_length_virginica = stats.levene(pl_virginica, sl_virginica, center='trimmed') # For heavy-tailed distributions
print(levene_length_virginica)

In [ ]:
# Fligner-Killeen's test: Non-parametric
fk_length_setosa = stats.fligner(pl_virginica, sl_virginica, center='mean') # For normal distributions
                                                                            # Use 'median' for skewed distributions
print(fk_length_setosa)

### Comparing 2 samples of a continuous measure: Parametric tests   
*t*-tests

In [ ]:
# t-test of 2 independent samples
ttest_sw_set_ver = stats.ttest_ind(sw_setosa, sw_versicolor, equal_var=True, nan_policy='omit') # equal_var default: T
print(ttest_sw_set_ver)

In [ ]:
# t-test of paired samples
ttest_width_setosa = stats.ttest_rel(pw_setosa, sw_setosa, nan_policy='omit')
print(ttest_width_setosa)

In [ ]:
# t-test from descriptive statistics: mean, SD, n from each sample
ttest_pw_vir_ver = stats.ttest_ind_from_stats(20.06, 2.902, 50, 13.26, 1.977, 50, equal_var=False)
print(ttest_pw_vir_ver)

**Effect sizes**  
Cohen's *d*

*d* = $\frac{x_1 - x_2}{SDp}$  
  
*d*=0.2 small effect size, 0.5 medium, 0.8 large    
  
$SD_p$ (Pooled standard deviation) = $\sqrt[2]{\frac{(N_1-1)(SD_1^2)+(N_2-1)(SD_2^2)}{N_1+N_2-2}}$

#### Exercise  
What is the value of Cohen's *d*?

In [ ]:
# Calculate pooled STD
std_sw_set_ver = np.sqrt( ( (sw_setosa.size-1)*(sw_setosa.std()**2) + (sw_versicolor.size-1)*(sw_versicolor.std()**2) ) 
                         / (sw_setosa.size + sw_versicolor.size - 2) )
# Calculate Cohen's d
cohend_sw_set_ver = (sw_setosa.mean() - sw_versicolor.mean()) / std_sw_set_ver
print('Cohen\'s d: ', cohend_sw_set_ver) # d=0.2 small effect size, 0.5 medium, 0.8 large

Pearson's correlation coefficient can also be used as a measure of effect size (see below)

In [ ]:
1-stats.norm.cdf(ttest_pw_vir_ver[0]) # one-side p-value if I know the test value

In [ ]:
stats.norm.ppf(ttest_pw_vir_ver[1]) # What is the test value given the p-value

### Comparing 2 samples of a continuous measure: Non-Parametric tests 
Wilcoxon rank-sum  
Mann-Whitney U  
Wilcoxon

In [ ]:
# Wilcoxon rank-sum test: Can use if n < 20
wrk_sw_set_ver = stats.ranksums(sw_setosa, sw_versicolor)
print(wrk_sw_set_ver)

In [ ]:
# Mann-Whitney U test: More robust than Wilcoxon rank-sum, use if n > 20
mwu_sw_set_ver = stats.mannwhitneyu(sw_setosa, sw_versicolor, use_continuity=True, alternative='greater')
                                    # alternative options: 'less', 'two-sided'. 'None' is deprecated
print(mwu_sw_set_ver)

In [ ]:
# Wilcoxon test: For paired samples
wilcoxon_width_setosa = stats.wilcoxon(pw_setosa, sw_setosa, zero_method='wilcox', correction=False)
                        # zero_method is how zero-differences are handled. Options: 'pratt', 'zsplit'
                        # correction is if statistic is corrected towards the mean during calculation. Default: F
print(wilcoxon_width_setosa)

**Comparing multiple groups**  
ANOVA  
Kruskal-Wallis H

In [ ]:
# 1-way ANOVA: Parametric
anova_sw = stats.f_oneway(sw_setosa, sw_versicolor, sw_virginica)
print(anova_sw)

*What about post-*hoc* tests, DF, and other results?*  
Not available in the Scipy.stats implementation  
Use of linear regression with the **statsmodels** module allows access to some of that data  

In [ ]:
# Kruskal-Wallis H test: Non-parametric
kw_sw = stats.kruskal(sw_setosa, sw_versicolor, sw_virginica, nan_policy='omit')
print(kw_sw)

### Contingency Tables   
Chi square  
Fisher's exact test

**pd.crosstab(vector1, vector2)** creates a contingency table from two binary vectors

#### Exercise  
Create a contingency table from counts of big and small petal width and sepal width using the mean as cutoff

In [ ]:
mean_pw = dataset['Petal_Width'].mean()
mean_sw = dataset['Sepal_Width'].mean()

width_table = pd.crosstab(dataset.Petal_Width > mean_pw, dataset.Sepal_Width > mean_sw)
width_table

In [ ]:
# Chi square: Requires a matrix composed of individual arrays or a pd.crosstab result as input 
chi2_width = stats.chi2_contingency(width_table, correction=False) # Correction: Yates'
                                     # Another optional argument: lambda_='pearson'/'log-likelihood'/'freeman-tukey'/
                                     # 'mod-log-likelihood'/'neyman'/'cressie-read'
                                     # lambda_ default is None which computes Pearson's chi2
print(chi2_width)
print('\n')
print(' Chi-square value: ', chi2_width[0], '\n',
      'p-value: ', chi2_width[1], '\n',
      'Degrees of freedom: ', chi2_width[2], '\n',
      'Expected frequencies: ', chi2_width[3], '\n')

In [ ]:
# Fisher's exact test: Use if any expected frequency is < 5
fisher_width = stats.fisher_exact([[18,42],
                                   [65,25]], alternative='two-sided') # alternative options: 'less', 'greater'
print(fisher_width)
print('\n')
print(' Odds ratio: ', fisher_width[0], '\n',
      'p-value: ', fisher_width[1])

### Correlation  
Pearson's correlation coefficient *r*  
Spearman rank-order correlation coefficient *rho*  
Point-biserial correlation coefficient  
Kendall's *Tau*

In [ ]:
# Pearson correlation coefficient: Parametric
pearson_petal = stats.pearsonr(dataset['Petal_Width'], dataset['Petal_Length'])
print(pearson_petal,'\n')
print('Pearson\'s correlation coefficient: ', pearson_petal[0])
print('p-value: ', pearson_petal[1]) # p-value is not so useful or reliable

In [ ]:
# Spearman rank-order correlation coefficient: Non-parametric
spearman_sepal = stats.spearmanr(dataset['Sepal_Width'], dataset['Sepal_Length'], nan_policy='omit')
print(spearman_sepal)

In [ ]:
# Point-biserial correlation coefficient: Measures correlation between a binary and a continuous variable
setosa_type = dataset['Type']=='setosa' #Binary variable
pbs_setosa_sw = stats.pointbiserialr(setosa_type, dataset['Sepal_Width'])
print(pbs_setosa_sw)

In [ ]:
# Kendall's Tau: Non-parametric. Arguments for use: Ordinal data, more robust than Spearman, non-linear relations
ktau_versicolor = stats.kendalltau(pw_versicolor, pl_versicolor, initial_lexsort=None, nan_policy='omit') 
                  # initial_lexsort=False uses quicksort
print(ktau_versicolor)

### Linear Regression

In [ ]:
# Scatterplot of variables to include in regression
sns.lmplot(y='Petal_Width', x='Sepal_Width', data=dataset) # Add hue='Type' to observe subgroups

In [ ]:
# Scipy linear regression using least-squares. Only works for univariate
scipy_linreg_width = stats.linregress(dataset['Sepal_Width'], dataset['Petal_Width']) # order of x,y != from lmplot
print(scipy_linreg_width)

**stats.linregress** provides limited information, and the library lacks a logistic regression function.  
Use the **statsmodels** library for regression

In [ ]:
import statsmodels.formula.api as smf 

In [ ]:
reg_width = smf.ols(formula='Petal_Width ~ Sepal_Width', data=dataset)
reg_width_model = reg_width.fit()
reg_width_model.summary()

In [ ]:
print(reg_width_model.summary()) # This way is better to obtain warnings

In [ ]:
print( dir(reg_width_model) )

To plot residuals, an important quality control step, we need to use the predict() method. This function takes as input a matrix of predictive variables plus a new column for the intercept. To create this compound matrix we need to use the add_constant() function from the other statsmodels api: **statsmodels.api**

In [ ]:
import statsmodels.api as sm

pred_var_matrix = dataset['Sepal_Width']
pred_var_matrix = sm.add_constant(pred_var_matrix)
sm_reg_width = sm.OLS( dataset['Petal_Width'], pred_var_matrix) # Note the difference from smf.ols
sm_reg_width_model = sm_reg_width.fit()

In [ ]:
# Plotting residuals:
# Obtain predicted values for dependent variable
predicted_values = reg_width_model.predict(pred_var_matrix) # dataset['Sepal_Width'] is not valid input
sm_predicted_values = sm_reg_width_model.predict(pred_var_matrix)

residuals = dataset['Petal_Width'] - sm_predicted_values
normalized_residuals = (residuals - np.mean(residuals)) / np.std(residuals)
normalized_predicted = (sm_predicted_values - np.mean(sm_predicted_values)) / np.std(sm_predicted_values)

plt.plot(normalized_residuals, normalized_predicted, 'o')
plt.xlabel('Standardized Residuals')
plt.ylabel('Standardized Predicted Value')

In [ ]:
influence = sm_reg_width_model.get_influence()
influence_dbetas = influence.summary_frame().filter(regex='dfb')
print(influence_dbetas.head(5))

DFBeta measures the influence a given sample exerts over the model. The maximum value allowed can be calculated as:  
$2^{\sqrt{N}}$

#### Exercise  
Find if any sample exerts an excessive influence over our model

In [ ]:
influence_max = 2**(np.sqrt(sm_reg_width_model.nobs)) 
print('Maximum value of DFBeta: ', influence_max)
any(influence_dbetas['dfb_Sepal_Width'] > influence_max)

### Logistic Regression

In [ ]:
logregr_setosa_sw = sm.Logit(setosa_type, pred_var_matrix)
logregr_setosa_sw_model = logregr_setosa_sw.fit()
print(logregr_setosa_sw_model.summary())

In [ ]:
# Calculate odds ratio
print(np.exp(logregr_setosa_sw_model.params))

#### Exercise  
Plot the data against the values predicted by the model

In [ ]:
# Plot predicted values vs. data
logregr_predicted_values = logregr_setosa_sw_model.predict(pred_var_matrix)
plt.plot(dataset['Sepal_Width'], setosa_type, 'o')
plt.plot(dataset['Sepal_Width'], logregr_predicted_values,'ok')
plt.xlabel('Sepal Width')
plt.ylabel('Setosa type')
plt.ylim(-0.05, 1.05)